# Neural Data Processing Notebook
### Kyle McGraw, February 2025

A notebook to explore a publicly available dataset on [neural population dynamics during reaching](https://dandiarchive.org/dandiset/000070)

## Loading the data